In [160]:
import scipy.io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

from skimage import io, color, filters, feature, restoration
from skimage.transform import resize, rotate
from skimage.color import rgb2gray
from sklearn.cluster import KMeans

In [161]:
mat = scipy.io.loadmat('../data/sat-6-full.mat')

In [162]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'train_x', 'test_x', 'train_y', 'test_y', 'annotations'])

In [176]:
x_train = mat.get('train_x')
y_train = mat.get('train_y')
x_test = mat.get('test_x')
y_test = mat.get('test_y')
ann = mat.get('annotations')
ann

array([[array(['100000'], dtype='<U6'), array(['building'], dtype='<U8')],
       [array(['010000'], dtype='<U6'),
        array(['barren land'], dtype='<U11')],
       [array(['001000'], dtype='<U6'), array(['trees'], dtype='<U5')],
       [array(['000100'], dtype='<U6'),
        array(['grassland'], dtype='<U9')],
       [array(['000010'], dtype='<U6'), array(['road'], dtype='<U4')],
       [array(['000001'], dtype='<U6'), array(['water'], dtype='<U5')]],
      dtype=object)

In [164]:
y_train.shape

(6, 324000)

In [165]:
x_train_play = x_train[:,:,:,0:1000]
y_train_play = y_train[:,0:1000]
x_test_play = x_test[:,:,:,0:1000]
y_test_play = y_test[:,0:1000]

In [184]:
# for i in range(test):
#     data = train_x_play[:,:,:,i]
#     img = Image.fromarray(data, 'RGBA')
#     img.save("../data/train_x/img{}.png".format(i))

def create_classifier_list(y_array):
    '''
    Creates a list of classes for each corresponding image in the X population:
    0 = building
    1 = barren land
    2 = trees
    3 = grassland
    4 = road
    5 = water
    '''
    
    i = 0
    class_list = []
    while i < len(y_array[1]):
        class_cat = np.where(y_array[:,i]==1)
        class_list.append(int(class_cat[0]))
        i+=1
    return class_list

# def sort_save_x(x_array):
#     '''
#     Saves 28x28x4 images into corresponding classification folders
#     '''

i = 0
for i in range(len(x_train_play[0,0,0,:])):
    data = x_train_play[:,:,:,i]
    img = Image.fromarray(data,'RGBA')
   
    if cls[i] == 0:
        img.save("../data/train_x/0_building/img{}.jpg".format(i))
    elif cls[i] == 1:
        img.save("../data/train_x/1_barren_land/img{}.jpg".format(i))
    elif cls[i] == 2:
        img.save("../data/train_x/2_trees/img{}.jpg".format(i))
    elif cls[i] == 3:
        img.save("../data/train_x/3_grassland/img{}.jpg".format(i))
    elif cls[i] == 4:
        img.save("../data/train_x/4_road/img{}.jpg".format(i))
    else:
        img.save("../data/train_x/5_water/img{}.jpg".format(i))
    
    
#     data = train_x_play[:,:,:,i]
#     img = Image.fromarray(data, 'RGBA')
#     img.save("../data/train_x/img{}.png".format(i))
            

324000

array([0, 1, 0, 0, 0, 0], dtype=uint8)